# Imports

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import pandas as pd
import os

from ipynb.fs.full.Neutron_Star_SPH_sph import *;
from ipynb.fs.full.Neutron_Star_SPH_eos import *;
from ipynb.fs.full.Neutron_Star_SPH_NS import NS,get_file_info;

# __Simulate NS__
Simulates the motion of the points for a single timestep, dt using the lagrangian formulation and eos.

#### Inputs: 
- eos_tag
- dt
- tEnd
- m
- r
- points_
- h_
- nu_
- lt
- kt
- date
- trial


- notes
- specs
- folder_name
- eta_
- cl

In [7]:
def simulate_NS(eos_tag,dt,tEnd,m,r,points_,nu_,lt,kt,date,trial,notes="",specs = "",folder_name = "Noname",eta_=1,cl=1,g_test_=6.65e-20):
    """ 
    # Simulate Motion of the points WRT THE STAR'S COM USING LAGRANGIAN
    """
    eos = EOS(eos_tag) #========= INITIALIZE Star and EOS
    NS_ = NS("mystar", eos,mass = m,radius = r,num_points = points_,nu=nu_,lagrange_tag=lt,kernel_type=kt,eta=eta_,clmbda=cl,g_test=g_test_) # INITIALIZE NS
    
    star1_com = [0,0,0] #========= Star COM Motion
    Nsteps = int(tEnd/dt)
    star1_list = [star1_com for i in range(int(Nsteps))] # This is a placeholder value and should come from the above block
    star1_pos = NS_.getPosition() # get position of stars currently
    a = np.asarray(star1_pos) # start off array of points with just the initial star locations.
    
    for t in range(int(Nsteps)): # for every timestep in the simulation
        pos1 = NS_.update(dt) # Update the star using the given timestep
        a = np.concatenate((a,pos1)) # Add SPH point positions for this timestep to the outputed data array
    
    # ===================== Animation dataframe ======================================= # 
    points_per_timestep = NS_.num_points # We need to assign the first set of points, (num_points1 + num_points2) points to the same timestep in the movie
    points_count = len(a) # Total number of points in the entire simulation
    t = np.array([np.ones(points_per_timestep)*i for i in range(int(points_count/points_per_timestep))]).flatten() # an array counting from 0 to 19. Each number repeats 100 times.
    df = pd.DataFrame({"time": t ,"x" : a[:,0], "y" : a[:,1], "z" : a[:,2]}) 
    print("\nSucessfully made plotting dataframe: \n")
    # ===================== ANIMATION ================================================= # 
    def update_graph(num):
        data=df[df['time']==num]
        graph._offsets3d = (data.x, data.y, data.z)
        title.set_text('NS Test, time={}'.format(num)) 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    title = ax.set_title('NS Test')
    data=df[df['time']==0]
    graph = ax.scatter(data.x, data.y, data.z)
    ani = matplotlib.animation.FuncAnimation(fig, update_graph, int(Nsteps), interval=10, blit=False)
    print("Creating file location...")
    path_to_file =  "test_results/" + folder_name + "/"#+ date + "/" + folder_name +trial # ======= SAVE ANIMATION RESULTS
    file_info = get_file_info(date, NS_, eos, dt, tEnd, notes) 
    try: 
        os.mkdir(path_to_file) # Make a new filepath to store the results.
    except:
        pass
    with open(path_to_file + "/" + 'info' +"test_" +specs + trial + '.txt', 'w') as f:
        f.write(file_info)
    plt.show() 
    print("Saving animation...")
    ani.save(path_to_file + "/"+"test_" + specs + trial + ".mp4") # Save animation.
    print("DONE.")
    return ani
    # ================================================================================== # 

_______________________________________________________________
# Parametric Testng (below)

 ### MULTIPLE PARAMETERS (AUTOMATIC) ### 
 
 ###### This will automate testing various parameters so you can just look at the output results.



In [5]:
eos_tag = "standard"
dt = 10 # seconds
tEnd = 6000 # seconds

M = [3] # SMs
R =[10] # km
Points_ = [50] # sph points
Nu_ = [4] # artificial viscosity
lt = "vanilla" # Lagrangian formulation tag
kt = "wendland_2" # Kernel type

for m in M:
    for r in R:
        for points_ in Points_:
            for nu_ in Nu_:
                break
                # Animation File Specifications 
                date = "Apr22"
                trial = "0"
                notes = "Morning of Apr 22. Testing various parameters with automated output. This is the initial test."
                spec = "test_params1"
                folder1 = "TEST_PARAMS2"
                folder2 = "r=" + str(r) + ",m=" + str(m) + ",str(nu_)" + str(nu)
                folder = folder1 + "/" + folder2 ## Save animation and output a file containing info about the animation.
                simulate_NS(eos_tag,dt,tEnd,m,r,points_,nu_,lt,kt,date,trial,notes=notes,specs = spec,folder_name=folder)

### SINGLE PARAMETERS SET (AUTOMATIC) ### 
 
 ###### This just tests the function with a single set of parameters



In [ ]:
###------------------------------------------------######------------------------------------------------###
 ### SINGLE SIMU
eos_tag = "standard"
dt = 10 # seconds
tEnd = 6000 # seconds
m = 10 # SMs
r = 3 # km
points_ = 300 # sph points
nu_ = 10 # artificial viscosity
lt = "vanilla" # Lagrangian formulation tag
kt = "wendland_2" # Gaussian 


# Animation File Specifications 
date = "Apr22"
trial = "0"
notes = "Altered the smoothing length."
spec = "altered_h"
folder1 = "test_smoothing_length1"
folder2 = "small_r/r=" + str(r) + ", m=" + str(m)

## Save animation and output a file containing info about the animation.

folder = folder1 + "/" + folder2
# simulate_NS(eos_tag,dt,tEnd,m,r,points_,nu_,lt,kt,date,trial,notes=notes,specs = spec,folder_name=folder)

###------------------------------------------------######------------------------------------------------###